In [3]:
include("../scripts/disc_solvers.jl");
using HDF5

In [5]:
data = h5read("../data/disc/pantagruel.h5","/");

In [6]:
id = Int64.(data["branch"][:,1:2])
b = 1.0 ./ data["branch"][:,4]
nline = length(b)


l = data["bus"][:,3] / 100.0
nb = length(l)

is_producing = data["gen"][:,2] .> 0
id_gen = Int64.(data["gen"][is_producing, 1])
id_load = setdiff(1:length(l), id_gen)
ng = length(id_gen)
nl = length(id_load)

g = zeros(nb)
g[id_gen] = data["gen"][is_producing,2] / 100.0
p = - l + g
p .-= sum(p) / nb

og = zeros(ng)

edges = Int64.(zeros(nline, 2))
line_start = data["branch"][:, 1]
line_end = data["branch"][:, 2]
# bus reordering: generator buses first 
for i in 1:ng
    edges[line_start .== id_gen[i], 1] .= i
    edges[line_end .== id_gen[i], 2] .= i
end
for i in 1:nl
    edges[line_start .== id_load[i], 1] .= i + ng
    edges[line_end .== id_load[i], 2] .= i + ng
end

id = edges
inc = sparse([id[:,1]; id[:,2]], [1:nline; 1:nline], [-ones(nline);ones(nline)])
println(length(id_gen))
# set initial conditions
og = zeros(ng)
thg = vec(data["bus"][id_gen, 9] / 180.0 * pi)
thl = vec(data["bus"][id_load, 9] / 180.0 * pi)
mg = vec(data["gen_inertia"][is_producing])
dg = vec(data["gen_prim_ctrl"][is_producing])
dload = vec(data["load_freq_coef"])
dg += dload[id_gen]
dl = dload[id_load]
pg = p[id_gen]
pl = p[id_load]
p = [pg; pl];

557


In [7]:
# get the stable solution
b2 = - im ./ data["branch"][:, 4]
Ybus = conj(inc * sparse(1:nline, 1:nline, b2) * inc')
q = zeros(nb);
V = ones(nb);
theta = zeros(nb);
include("../scripts/disc_solvers.jl")
V, theta, iter = NRsolver(Ybus, V, theta, -p, q, Array{Int64,1}([]), 9, tol = 1E-30, maxiter = 10)
thg = theta[1:ng]
thl = theta[ng+1:end];
og = zeros(ng);

Max iteration reached, error: 2.128242027055194e-12


In [8]:
dt = 1e-3
Ndt = 5e4
interval = 50
ts = zeros(Int64.(floor(Ndt / interval))+1)
omegas = zeros(ng, Int64.(floor(Ndt / interval))+1)
k = 1
dp = zeros(ng)
dp[1] = -9.0
for i in 1:Ndt
    y = radau5(og, thg, thl, mg, dg, dl, pg+dp, pl, inc, b, dt, maxiter = 14, tol = 1E-6)
    if(mod(i, interval) == 0)
        println(i, " / ", Ndt, " (", floor(100*i/Ndt), "%)")
            omegas[:, k] = y[1 : ng]
            ts[k] = i*dt
            k += 1
        end
    og = y[1:ng]
    thg = y[ng + 1 : 2 * ng]
    thl = y[2 * ng + 1 : end]
end
    

50.0 / 50000.0 (0.0%)
100.0 / 50000.0 (0.0%)
150.0 / 50000.0 (0.0%)
200.0 / 50000.0 (0.0%)
250.0 / 50000.0 (0.0%)
300.0 / 50000.0 (0.0%)
350.0 / 50000.0 (0.0%)
400.0 / 50000.0 (0.0%)
450.0 / 50000.0 (0.0%)
500.0 / 50000.0 (1.0%)
550.0 / 50000.0 (1.0%)
600.0 / 50000.0 (1.0%)
650.0 / 50000.0 (1.0%)
700.0 / 50000.0 (1.0%)
750.0 / 50000.0 (1.0%)
800.0 / 50000.0 (1.0%)
850.0 / 50000.0 (1.0%)
900.0 / 50000.0 (1.0%)
950.0 / 50000.0 (1.0%)
1000.0 / 50000.0 (2.0%)
1050.0 / 50000.0 (2.0%)
1100.0 / 50000.0 (2.0%)
1150.0 / 50000.0 (2.0%)
1200.0 / 50000.0 (2.0%)
1250.0 / 50000.0 (2.0%)
1300.0 / 50000.0 (2.0%)
1350.0 / 50000.0 (2.0%)
1400.0 / 50000.0 (2.0%)
1450.0 / 50000.0 (2.0%)
1500.0 / 50000.0 (3.0%)
1550.0 / 50000.0 (3.0%)
1600.0 / 50000.0 (3.0%)
1650.0 / 50000.0 (3.0%)
1700.0 / 50000.0 (3.0%)
1750.0 / 50000.0 (3.0%)
1800.0 / 50000.0 (3.0%)
1850.0 / 50000.0 (3.0%)
1900.0 / 50000.0 (3.0%)
1950.0 / 50000.0 (3.0%)
2000.0 / 50000.0 (4.0%)
2050.0 / 50000.0 (4.0%)
2100.0 / 50000.0 (4.0%)
2150.0 / 500

16450.0 / 50000.0 (32.0%)
16500.0 / 50000.0 (33.0%)
16550.0 / 50000.0 (33.0%)
16600.0 / 50000.0 (33.0%)
16650.0 / 50000.0 (33.0%)
16700.0 / 50000.0 (33.0%)
16750.0 / 50000.0 (33.0%)
16800.0 / 50000.0 (33.0%)
16850.0 / 50000.0 (33.0%)
16900.0 / 50000.0 (33.0%)
16950.0 / 50000.0 (33.0%)
17000.0 / 50000.0 (34.0%)
17050.0 / 50000.0 (34.0%)
17100.0 / 50000.0 (34.0%)
17150.0 / 50000.0 (34.0%)
17200.0 / 50000.0 (34.0%)
17250.0 / 50000.0 (34.0%)
17300.0 / 50000.0 (34.0%)
17350.0 / 50000.0 (34.0%)
17400.0 / 50000.0 (34.0%)
17450.0 / 50000.0 (34.0%)
17500.0 / 50000.0 (35.0%)
17550.0 / 50000.0 (35.0%)
17600.0 / 50000.0 (35.0%)
17650.0 / 50000.0 (35.0%)
17700.0 / 50000.0 (35.0%)
17750.0 / 50000.0 (35.0%)
17800.0 / 50000.0 (35.0%)
17850.0 / 50000.0 (35.0%)
17900.0 / 50000.0 (35.0%)
17950.0 / 50000.0 (35.0%)
18000.0 / 50000.0 (36.0%)
18050.0 / 50000.0 (36.0%)
18100.0 / 50000.0 (36.0%)
18150.0 / 50000.0 (36.0%)
18200.0 / 50000.0 (36.0%)
18250.0 / 50000.0 (36.0%)
18300.0 / 50000.0 (36.0%)
18350.0 / 50

48000.0 / 50000.0 (96.0%)
48050.0 / 50000.0 (96.0%)
48100.0 / 50000.0 (96.0%)
48150.0 / 50000.0 (96.0%)
48200.0 / 50000.0 (96.0%)
48250.0 / 50000.0 (96.0%)
48300.0 / 50000.0 (96.0%)
48350.0 / 50000.0 (96.0%)
48400.0 / 50000.0 (96.0%)
48450.0 / 50000.0 (96.0%)
48500.0 / 50000.0 (97.0%)
48550.0 / 50000.0 (97.0%)
48600.0 / 50000.0 (97.0%)
48650.0 / 50000.0 (97.0%)
48700.0 / 50000.0 (97.0%)
48750.0 / 50000.0 (97.0%)
48800.0 / 50000.0 (97.0%)
48850.0 / 50000.0 (97.0%)
48900.0 / 50000.0 (97.0%)
48950.0 / 50000.0 (97.0%)
49000.0 / 50000.0 (98.0%)
49050.0 / 50000.0 (98.0%)
49100.0 / 50000.0 (98.0%)
49150.0 / 50000.0 (98.0%)
49200.0 / 50000.0 (98.0%)
49250.0 / 50000.0 (98.0%)
49300.0 / 50000.0 (98.0%)
49350.0 / 50000.0 (98.0%)
49400.0 / 50000.0 (98.0%)
49450.0 / 50000.0 (98.0%)
49500.0 / 50000.0 (99.0%)
49550.0 / 50000.0 (99.0%)
49600.0 / 50000.0 (99.0%)
49650.0 / 50000.0 (99.0%)
49700.0 / 50000.0 (99.0%)
49750.0 / 50000.0 (99.0%)
49800.0 / 50000.0 (99.0%)
49850.0 / 50000.0 (99.0%)
49900.0 / 50

In [11]:
fid = h5open("../numerics/disc_results.h5", "w")
write(fid, "ts", ts)
write(fid, "omegas", omegas)
close(fid)